In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss, f1_score, precision_score, recall_score

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# import nltk
# nltk.download('punkt')

In [5]:
questions_df = pd.read_csv('Questions.csv', encoding = "ISO-8859-1")
answers_df = pd.read_csv('Answers.csv', encoding = "ISO-8859-1")
tags_df = pd.read_csv('Tags.csv')

In [7]:
questions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [9]:
answers_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014516 entries, 0 to 2014515
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            2014516 non-null  int64  
 1   OwnerUserId   2001316 non-null  float64
 2   CreationDate  2014516 non-null  object 
 3   ParentId      2014516 non-null  int64  
 4   Score         2014516 non-null  int64  
 5   Body          2014516 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 92.2+ MB


In [10]:
tags_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Id      3750994 non-null  int64 
 1   Tag     3749881 non-null  object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [11]:
answers_df

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."
...,...,...,...,...,...,...
2014511,40143247,333403.0,2016-10-19T23:42:35Z,40143190,0,"<p>Tanks to <a href=""http://stackoverflow.com/..."
2014512,40143322,642706.0,2016-10-19T23:50:35Z,40137110,1,<h1>tl;dr</h1>\n\n<pre><code>ZonedDateTime.par...
2014513,40143336,2239781.0,2016-10-19T23:52:08Z,40141860,0,<p>I came up with a very dirty workaround. Bef...
2014514,40143349,6934347.0,2016-10-19T23:54:02Z,40077010,0,<p>I solved my own problem defining the follow...


In [13]:
len(list(answers_df.Id.unique()))

2014516

In [14]:
questions_df.drop(['OwnerUserId', 'CreationDate', 'ClosedDate', 'Score'], axis=1, inplace=True)

In [15]:
answers_df.drop(['Id', 'OwnerUserId','CreationDate', 'Score'], axis=1, inplace=True)

In [16]:
answers_df.rename(columns = {'ParentId':'Id'}, inplace = True)

In [18]:
merged_df = questions_df.merge(answers_df, on='Id').merge(tags_df, on='Id')

merged_df.rename(columns = {'Body_x':'question_text'}, inplace = True)

merged_df.rename(columns = {'Body_y':'answer_text'}, inplace = True)

In [24]:
len(tags_df.Tag.unique())

37035

In [20]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text
    words = word_tokenize(text)
    
    # Remove punctuation and stopwords
    words = [word for word in words if word.isalnum() and word not in stopwords.words('english')]
    
    # Join the processed words back into a single string
    processed_text = ' '.join(words)
    
    return processed_text

In [21]:
# Preprocess text data (e.g., remove punctuation, lowercase)
merged_df['question_text'] = merged_df['question_text'].apply(preprocess_text)

x = merged_df['question_text']
y = merged_df['tags']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\vscse/nltk_data'
    - 'C:\\Users\\vscse\\anaconda3\\nltk_data'
    - 'C:\\Users\\vscse\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\vscse\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\vscse\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

mlb = MultiLabelBinarizer()
y_train_encoded = mlb.fit_transform(y_train)
y_test_encoded = mlb.transform(y_test)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vscse\AppData\Roaming\nltk_data...
[nltk_data] Error downloading 'punkt' from
[nltk_data]     <https://raw.githubusercontent.com/nltk/nltk_data/gh-
[nltk_data]     pages/packages/tokenizers/punkt.zip>:   <urlopen error
[nltk_data]     [Errno 2] No such file or directory>


False

In [ ]:
# Train a Binary Relevance model with Logistic Regression as the base classifier
base_classifier = LogisticRegression(solver='liblinear')
classifier = MultiOutputClassifier(base_classifier)
classifier.fit(X_train_tfidf, y_train_encoded)

In [ ]:
# Predict multiple tags for test data
y_pred_encoded = classifier.predict(X_test_tfidf)

# Inverse transform the predicted tags
y_pred_tags = mlb.inverse_transform(y_pred_encoded)

In [ ]:
# Evaluate the model
hamming_loss_score = hamming_loss(y_test_encoded, y_pred_encoded)
f1 = f1_score(y_test_encoded, y_pred_encoded, average='micro')
precision = precision_score(y_test_encoded, y_pred_encoded, average='micro')
recall = recall_score(y_test_encoded, y_pred_encoded, average='micro')

print(f'Hamming Loss: {hamming_loss_score}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')